In [ ]:
import random, re, json, pathlib, logging, time, argparse, pprint 
import pandas as pd
import numpy as np
import circlify as crcf
import matplotlib.pyplot as plt
import importlib.resources as rsrc

import postprocess.postprocess as pspr


In [ ]:
target_df = pd.read_pickle("gd_target_df.pkl")

goal_df = pd.read_pickle("gd_goal_df.pkl")

In [ ]:
goal_df.head()

In [ ]:

target_df['Target'] = pspr.stringify_id(target_df['Target'])

results_dict = {}

with rsrc.path("postprocess", "goal_target_list.xlsx") as res_path:
    sdg_df = pd.read_excel(res_path)

results_dict['target_dat'] = pspr.aggregate_to_targets(target_df, sdg_df)

results_dict['goal_dat'] = pspr.aggregate_to_goals(goal_df, sdg_df)

results_dict['goals_grouped_by_document'] = pspr.group_byNAme_and_get_goaloverview(results_dict['target_dat'], results_dict['goal_dat'], sdg_df)

In [ ]:
# results_dict['goals_grouped_by_document']=results_dict['goals_grouped_by_document'].drop(columns = results_dict['goals_grouped_by_document'].columns[[0]], inplace=True)

In [ ]:
results_dict['goals_grouped_by_document']=results_dict['goals_grouped_by_document'].rename_axis(['names',0])

In [ ]:
results_dict['goals_grouped_by_document'].groupby("Policy").groups
#results_dict['goals_grouped_by_document'].groups 

In [ ]:
feh = results_dict['goals_grouped_by_document'][['Goal','Count']].to_dict('index')
print(feh)

In [ ]:
for name in results_dict['goals_grouped_by_document']:
    print(name)


In [ ]:
circles = crcf.circlify(
    results_dict['goals_grouped_by_document'][['Goal','Count']].to_dict('index'), 
    show_enclosure=False, 
    target_enclosure=crcf.Circle(x=0, y=0, r=1)
)